In [1]:
import cv2 as cv
import numpy as np
import os

In [2]:
classes = []
file_path = "coco.names"  # Use the correct relative or absolute path
with open(file_path, "r") as f:
    classes = [line.strip() for line in f.readlines()]

In [3]:
execution_path = os.getcwd()
yolo_net = cv.dnn.readNet(os.path.join(execution_path,"yolov3.weights"), os.path.join(execution_path,"yolov3.cfg"))

In [4]:
layer_names = yolo_net.getLayerNames()
output_layers = [layer_names[i - 1] for i in yolo_net.getUnconnectedOutLayers()]


In [5]:
image = cv.imread("04_tehran.png")
height, width, channels = image.shape

In [6]:
img_blob = cv.dnn.blobFromImage(image, 0.00392, (416, 416), (0, 0, 0), True, crop=False)

In [7]:
yolo_net.setInput(img_blob)
outputs = yolo_net.forward(output_layers)
outputs[0].shape

(507, 85)

In [8]:
class_ids = []
confidences = []
boxes = []
for out in outputs:
    for detection in out:
        scores = detection[5:]
        class_id = np.argmax(scores)
        confidence = scores[class_id]
        if confidence> 0.5:
            center_x = int(detection[0] * width)
            center_y = int(detection[1] * height)
            w = int(detection[2] * width)
            h = int(detection[3] * height)

            # Rectangle coordinates
            x = int(center_x - w / 2)
            y = int(center_y - h / 2)
            boxes.append([x, y, w, h])
            confidences.append(float(confidence))
            class_ids.append(class_id)

In [9]:


colors = np.random.uniform(0, 255, size=(len(classes), 3))
colors[0],colors[1],colors[2],colors[5],colors[9] = (0, 136, 255), (226, 43, 138), (255, 191, 0), (153, 255, 255), (164, 203, 255)
indexes = cv.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

for i in range(len(boxes)):
    if i in indexes:
        x, y, w, h = boxes[i]
        label = str(classes[class_ids[i]])
        color = colors[class_ids[i]]
        cv.rectangle(image, (x, y), (x + w, y + h), color, 2)
        cv.putText(image, label, (x, y + 30), cv.FONT_HERSHEY_PLAIN, 1, color, 2)

cv.imwrite("05_tehran_det_cv.png",image)

True

In [11]:


cap = cv.VideoCapture("driving_camera.mp4")

while(cap.isOpened()):
  ret, frame = cap.read()
  if ret == True:
    img = cv.resize(frame, None, fx=0.8, fy=0.8)
    height, width, channels = img.shape
    blob = cv.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    yolo_net.setInput(blob)
    outs = yolo_net.forward(output_layers)
    class_ids = []
    confidences = []
    boxes = []
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:
                # Object detected
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)

                # Rectangle coordinates
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)

                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    indexes = cv.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
    print(indexes)
    font = cv.FONT_HERSHEY_PLAIN
    for i in range(len(boxes)):
        if i in indexes:
            x, y, w, h = boxes[i]
            label = str(classes[class_ids[i]])
            color = colors[class_ids[i]]
            cv.rectangle(img, (x, y), (x + w, y + h), color, 2)
            cv.putText(img, label, (x, y + 30), font, 2, color, 2)
    # cv.imshow('Frame',img)
    # if cv.waitKey(25) & 0xFF == ord('q'):
    #   break
# cap = cv.VideoCapture("driving_camera.mp4")


cv.VideoWriter("driving_camera.mp4", fourcc, fps, frameSize[, isColor])

# cap.release()
# cv.destroyAllWindows()

SyntaxError: invalid syntax (1114202960.py, line 50)